# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686609


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:24,  3.01s/it]

Rendering models:  23%|██▎       | 7/30 [00:06<00:48,  2.11s/it]

Rendering models:  30%|███       | 9/30 [00:06<00:31,  1.49s/it]

Rendering models:  40%|████      | 12/30 [00:06<00:19,  1.06s/it]

...

Rendering models:  70%|███████   | 21/30 [00:06<00:03,  2.59it/s]

Rendering models:  80%|████████  | 24/30 [00:06<00:01,  3.53it/s]

Rendering models:  90%|█████████ | 27/30 [00:06<00:00,  4.76it/s]

Rendering models: 100%|██████████| 30/30 [00:07<00:00,  6.33it/s]

cschwefl                               0.045223
Dynamatt                               0.000262
asucich                                0.000225
djswanso                               0.073251
thedarktree                            0.000272
not-logged-in-a7f3c0eb6d81d69e04a2     0.001765
acohen12                               0.000234
rnevils                                0.000326
carolynjoy                             0.000495
spider_pig                             0.002307
pangeli5                               0.000621
froggler                               0.000266
Lavadude                               0.022944
hpatss1                                0.000218
Tessaje04                              0.000858
not-logged-in-36a6cd4ee9f5f5a4f610     0.000586
jmacd297                               0.001184
not-logged-in-3429675089c67c4fccb8     0.005079
cooper.hull                           67.903064
not-logged-in-ec641b5d49aaa15bc8a9     0.004052
not-logged-in-8250d4496dd19b9ab401     0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())